In [1]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import numpy as np
from config import _highest_cost_rate, _transaction_fee, SYMBOLS

In [2]:
dax_trading_results = pd.DataFrame(columns=["ticker","cash_balance", "share_holding", "asset", "transaction_cost"
                                            , "total_reward" ])

for ticker in SYMBOLS:
    df = yf.Ticker(ticker).history(start='2020-07-01', end='2021-10-01')[['Open','High','Low', 'Close']]
    df.ta.aroon(length=25, append=True)
    df['aroon_buy'] = ((df.AROONU_25.shift(1) < df.AROOND_25.shift(1)) & (df.AROONOSC_25 >= 0)).astype(int)
    df['aroon_sell'] = ((df.AROONU_25.shift(1) > df.AROOND_25.shift(1)) & (df.AROONOSC_25 <= 0)).astype(int)
    df.columns = [x.lower() for x in df.columns]
    test_data = df[df.index >= '2020-10-01']

    initial_balance = 1000000
    initial_shares = 0

    cash_balance = initial_balance
    share_holding = initial_shares
    yesterday_flag = 0
    trading_outlay = list()
    total_reward = 0
    total_transaction_cost = 0

    for row in test_data.iterrows():

        reward = 0
        transaction_price = (row[1].high+row[1].low)/2
        transaction_cost = 0

        if yesterday_flag == 1: #buy
            # the logic here is to assume I pay the highest fees, how many hands can I afford
            max_hand = np.floor((cash_balance)/(transaction_price*_highest_cost_rate('Frankfurt')))
            transaction_shares = max_hand

            transaction_amount = transaction_price*transaction_shares
            transaction_cost = _transaction_fee('Frankfurt',yesterday_flag,transaction_amount)

            cash_balance = cash_balance-transaction_amount-transaction_cost
            share_holding = share_holding+transaction_shares

            trading_outlay.append(transaction_cost+transaction_amount)

        elif yesterday_flag == -1: #sell
            transaction_shares = share_holding
            transaction_amount = transaction_price*transaction_shares

            transaction_cost = _transaction_fee('Frankfurt',yesterday_flag,transaction_amount)

            cash_balance = cash_balance+transaction_amount-transaction_cost
            share_holding = 0
            reward = transaction_amount-transaction_cost-np.sum(trading_outlay)
            trading_outlay = list()

        asset = round(cash_balance+share_holding*transaction_price,0)
        total_reward += reward
        total_transaction_cost += transaction_cost

        # observe today's trading signal and carry out tomorrow
        if row[1].aroon_buy == 1:
            yesterday_flag = 1
        elif row[1].aroon_sell == 1:
            yesterday_flag = -1
        else:
            yesterday_flag = 0

    dax_trading_results = dax_trading_results.append({"ticker":ticker,"cash_balance":round(cash_balance,2)
                                              ,"share_holding":share_holding, "asset":asset
                                              ,"transaction_cost":total_transaction_cost
                                              ,"total_reward":total_reward
                                             }, ignore_index=True)

dax_trading_results.to_excel("DAX_trading_results.xlsx")